#  Исследование эффективности двух функций возведения в степень

Будем сравнвать две следующие функции:

In [283]:
def wtf(unicorn, pony):
  if pony == 1:
     return unicorn
  elif pony % 2 == 0:
    return wtf(unicorn * unicorn, pony // 2)
  elif pony % 2 == 1:
    return unicorn * wtf(unicorn * unicorn, pony // 2)

In [284]:
def power(num, deg):
  return num ** deg

Сравнение будем производить по следующим критериям:

1) Время работы

2) Используемая память

3) Применимость функции

## Время работы

Импортируем нужные библиотеки

In [285]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "presentation"

Исследуем время работы функций на разных входных данных, построим графики.

Для болшей точности будем измерять время на конкретном входе 100000 раз.

In [286]:
def compre_func_time_plot(grid, num):
  wtf_times = []
  power_times = []
  input_sizes = []

  for deg in grid:
    input_sizes.append(deg)

    wtf_time = timeit.timeit(f'wtf({num}, {deg})', globals=globals(), number=100000)
    wtf_times.append(wtf_time)

    power_time = timeit.timeit(f'power({num}, {deg})', globals=globals(), number=100000)
    power_times.append(power_time)

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=input_sizes, y=wtf_times,
                    mode='lines+markers',
                    name='wtf'))
  fig.add_trace(go.Scatter(x=input_sizes, y=power_times,
                    mode='lines+markers',
                    name='power'))
  fig.update_layout(
    title="Comparison of WTF and Power Times",
    xaxis_title="degree",
    yaxis_title="sec")
  fig.show()
  return wtf_times, power_times

Поcтроим графики для степеней двойки

In [287]:
wtf_times1, power_times1 = compre_func_time_plot(range(1, 500, 2), 2)

**Выводы:**

1) Сразу видно, что wtf существенно проигрывает по времени, во все степени кроме 1 wtf в среднем возводит дольше

2) Наклон тренда у функции wtf больше, с увелечением степени время выполнения растет быстрее чем у power



Также посмотрим поведение на больших степенях:

In [288]:
wtf_times2, power_times2 = compre_func_time_plot(range(500, 2000, 5), 2)

Ситуация в целом похожая

In [289]:
wtf_times = wtf_times1 + wtf_times2
power_times = power_times1 + power_times2
diff_times = []
for i in range(len(wtf_times)):
  diff_times.append(wtf_times[i] / power_times[i])

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(diff_times))), y=diff_times,
                    mode='lines+markers',
                    name='diffs'))

fig.update_layout(
    title="Difference between wtf and power",
    xaxis_title="iteration",
    yaxis_title="x-times wtf worse")
fig.show()

Посчитаем во сколько раз в среднем wtf медленее power

In [290]:
print(f"В среднем wtf медленее в {round(np.mean(diff_times), 2)} раза")

В среднем wtf медленее в 3.21 раза


## Используемая память

Сравнивая две функции с рекурсией и без, нельзя не задуматься о испоользуемой памяти. В случае рекурсии мы храним стек вызовов, проверим, влечет ли в этом случае это существенные расходы

In [291]:
from memory_profiler import memory_usage


def find_mem_usage(grid, num):
  wtf_mems = []
  power_mems = []
  input_sizes = []

  for deg in grid:
    input_sizes.append(deg)

    wtf_mem = memory_usage((wtf, (num, deg)))
    wtf_mems.append(max(wtf_mem) - min(wtf_mem))

    power_mem = memory_usage((power, (num, deg)))
    power_mems.append(max(power_mem) - min(power_mem))

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=input_sizes, y=wtf_mems,
                    mode='lines+markers',
                    name='wtf'))
  fig.add_trace(go.Scatter(x=input_sizes, y=power_mems,
                    mode='lines+markers',
                    name='power'))
  fig.update_layout(
    title="Comparison of WTF and Power Memory usage",
    xaxis_title="degree",
    yaxis_title="MiB")
  fig.show()
  return wtf_mems, power_mems

In [292]:
wtf_mems1, power_mems1 = find_mem_usage(range(1, 500, 2), 2)

**Вывод:**

Видим, что обе функции либо не тратят память, либо траты существенно малы

Проверим на всякий случай еще на больших степенях

In [293]:
wtf_mems2, power_mems2 = find_mem_usage(range(500, 10000, 100), 2)

На больших степенях расходы памяти тоже не видны

## Применимость функции

Тут можно выделить несколько недостатков функции wtf:

1) Функция не работает с отрицательными степенями

wtf(2, -1) - такой код приведет к бесконечной рекурсии

В то время как power работает:

In [294]:
power(2, -1)

0.5

2) Функция не работает с дробными степенями




In [295]:
print(wtf(2, 0.5))

None


При помощи power можно возводить в дробные степени

In [296]:
power(2, 0.5)

1.4142135623730951

## Выводы